In [1]:
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM

In [ ]:
assert torch.cuda.is_available()
device = torch.device("cuda")
print("Using device:", device)

In [ ]:
import json
with open('bruteforce2.json', 'r') as file:
    finetune_data = json.load(file)
finetune_data=finetune_data["data"]

In [ ]:
from transformers import AutoTokenizer
# Your code here
tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token # convenient for padding later

In [ ]:
from transformers import AutoModelForCausalLM
# Your code here
gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2').to(device)

In [ ]:
class CrypticDataset(Dataset):
    PROMPT = "Find the next state to solve the following Cryptic Crossword. DEFINITION {definition} LENGTH {length} CLUE {clue} | STEPS {steps} STATE {state}\n\n NEXTSTATE {next_state}"

    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        tokenizer.padding_side = 'right'

        self.input_ids = []
        self.attn_masks = []
        self.labels = []

        training_texts = []
        for example in self.data:
            steps = example["steps"]
            for i in range(len(steps)):
                training_text = CrypticDataset.PROMPT.format(definition=example['definition'], length=example["length"],
                                                             clue=example['clue'], state=steps[i], next_state=("<stop>" if i == len(example("steps"))-1 else steps[i+1])) + "<|endoftext|>" # include the end token so model knows when to stop!
                training_texts.append(training_text)
        encodings_dict = self.tokenizer(training_texts, padding=True, truncation=True)
        for i, (example, training_text) in enumerate(zip(data, training_texts)):
            self.input_ids.append(torch.tensor(encodings_dict['input_ids'][i]))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask'][i]))
            # Your code here
            prompt_and_input_length = len(tokenizer.encode(
                "Find the next state tosolve the following Cryptic Crossword. DEFINITION {definition} LENGTH {length} CLUE {clue} | STEPS {i} STATE {state}\n\n NEXTSTATE ".format(
                    definition=example['definition'], length=example["length"], clue=example['clue'], state=steps[i])))
            self.labels.append(torch.tensor([-100] * prompt_and_input_length + encodings_dict['input_ids'][i][prompt_and_input_length:]))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {'input_ids': self.input_ids[idx], 'attention_mask': self.attn_masks[idx], 'labels': self.labels[idx]}

In [ ]:
PROMPT = "Solve the following Cryptic Crossword: DEFINITION {definition} LENGTH {length} CLUE {clue} | STEPS {steps} STATE {state} NEXTSTATE {next_state}"